> 🩺 Este notebook forma parte del proyecto **NutriSynthCare**, un entorno de simulación sobre salud pública y recomendaciones nutricionales personalizadas.  
>
> 👥 Desarrollado por:  
> - Daniel Cruz – [dCruzCoding](https://github.com/dCruzCoding)  
> - Aníbal García – [Aniballll](https://github.com/Aniballll)  
>
> 📁 Repositorio completo: [NutriSynthCare](https://github.com/tu-repo-aqui)  
>
> 📜 Licencia: Este proyecto está disponible bajo la licencia MIT. Consulta el archivo `LICENSE` para más información.  
>
> 🙏 Agradecimientos especiales a todas las fuentes de datos y literatura científica que han hecho posible la creación de las bases sintéticas utilizadas en este proyecto.  
>
> 🤝 ¿Te interesa colaborar? ¡Estás invitado!  
> Puedes contribuir mejorando los notebooks, proponiendo nuevas ideas o corrigiendo errores:  
> - Haz un fork del repo  
> - Crea una rama (`git checkout -b mejora/nueva-idea`)  
> - Abre un Pull Request explicando tu propuesta


# Objetivo del Notebook

En este notebook se aborda la **desnormalización** de ciertas variables que previamente fueron normalizadas para facilitar la unión de db_cardio y db_diabetes. 

Cada cohorte (`cardio` y `diabetes`) contenía muestras distintas con sus propias distribuciones, por lo que la normalización se realizó por separado en cada grupo, utilizando sus propios parámetros (media y desviación estándar).

Para revertir esta normalización y volver a obtener los valores en sus escalas originales, se aplicará la desnormalización usando los parámetros específicos de cada cohorte. Esto permite trabajar con los datos integrados sin perder la interpretación clínica y estadística original de las variables.

Este proceso es clave para validar la correcta integración de los datos y para asegurar que los análisis posteriores se realicen sobre valores representativos y clínicamente interpretables.


In [13]:
import pandas as pd
import numpy as np

# Cargamos los csv y lo convertimos en dataframes
df = pd.read_csv("db_cardiabetes.csv")      # Dataset normalizado con cohortes mezcladas
df_cardio = pd.read_csv("db_cardio.csv")    # Dataset original cohorte cardio
df_diabetes = pd.read_csv("db_diabetes.csv")# Dataset original cohorte diabetes


In [14]:
from sklearn.preprocessing import StandardScaler 

# Columnas que se van a normalizar y desnormalizar
cols_to_normalize = ['Trigliceridos', 'PAS', 'PAD','Edad', 'IMC']

def normalizar_para_guardar_parametros(df, cols):
    """
    Normaliza las columnas indicadas usando StandardScaler y guarda los parámetros (media y std).
    Retorna el dataframe normalizado y un diccionario con los parámetros de escala.
    """
    scaler = StandardScaler()
    df_normalizado = df.copy()
    df_normalizado[cols] = df_normalizado[cols].astype('float64')  # Asegurar tipo float para escala
    df_normalizado.loc[:, cols] = scaler.fit_transform(df[cols])
    parametros = {'media': scaler.mean_, 'std': scaler.scale_}     # Guardamos media y desviación estándar
    return df_normalizado, parametros

# Aplicar normalización a cada cohorte por separado para obtener parámetros específicos
_, params_cardio = normalizar_para_guardar_parametros(df_cardio, cols_to_normalize)
_, params_diabetes = normalizar_para_guardar_parametros(df_diabetes, cols_to_normalize)

# Guardamos los parámetros en un diccionario por cohorte para uso posterior
params_norm = {
    'cardio': params_cardio,
    'diabetes': params_diabetes
}

In [15]:
def desnormalizar_con_standard(df, params_norm, cols_to_normalize):
    """
    Desnormaliza las columnas indicadas de un dataframe normalizado `df`
    usando los parámetros guardados `params_norm` (media y std) según cohorte.
    """
    df_desnormalizado = df.copy()
    
    # Para cada cohorte, desnormalizamos solo las filas que pertenecen a ella
    for cohorte in params_norm.keys():
        mask = df['Cohorte'] == cohorte
        for i, col in enumerate(cols_to_normalize):
            mean = params_norm[cohorte]['media'][i]
            std = params_norm[cohorte]['std'][i]
            # Aplicamos la fórmula inversa del StandardScaler: x_original = x_norm * std + mean
            df_desnormalizado.loc[mask, col] = (df[col][mask] * std) + mean
    
    return df_desnormalizado

# Aplicamos la desnormalización sobre el dataframe general con cohortes mezcladas
df_desnormalizado = desnormalizar_con_standard(df, params_norm, cols_to_normalize)

In [16]:
def verificar_desnormalizacion(df_desnormalizado, df_originales, cols):
    """
    Compara las estadísticas (media y std) entre el dataframe desnormalizado y
    los dataframes originales por cohorte para verificar que la desnormalización es correcta.
    """
    resultados = []
    for cohorte in ['cardio', 'diabetes']:
        mask = df_desnormalizado['Cohorte'] == cohorte
        for col in cols:
            original_mean = df_originales[cohorte][col].mean()
            original_std = df_originales[cohorte][col].std()
            desnorm_mean = df_desnormalizado[mask][col].mean()
            desnorm_std = df_desnormalizado[mask][col].std()
            
            resultados.append({
                'Variable': col,
                'Cohorte': cohorte,
                'Original_Mean': original_mean,
                'Desnorm_Mean': desnorm_mean,
                'Original_Std': original_std,
                'Desnorm_Std': desnorm_std
            })
    return pd.DataFrame(resultados)


In [17]:
def comparar_global(df, df_desnormalizado, variables):
    """
    Compara estadísticos globales (media, std, percentiles) entre el dataframe
    normalizado y el desnormalizado para las variables indicadas.
    """
    resultados = []
    
    for var in variables:
        # Estadísticas del DataFrame normalizado
        stats_norm = {
            'Variable': var,
            'Estado': 'Normalizado',
            'Media': df[var].mean(),
            'Std': df[var].std(),
            'Min': df[var].min(),
            'P25': df[var].quantile(0.25),
            'P50 (Mediana)': df[var].median(),
            'P75': df[var].quantile(0.75),
            'Max': df[var].max()
        }
        
        # Estadísticas del DataFrame desnormalizado
        stats_desnorm = {
            'Variable': var,
            'Estado': 'Desnormalizado',
            'Media': df_desnormalizado[var].mean(),
            'Std': df_desnormalizado[var].std(),
            'Min': df_desnormalizado[var].min(),
            'P25': df_desnormalizado[var].quantile(0.25),
            'P50 (Mediana)': df_desnormalizado[var].median(),
            'P75': df_desnormalizado[var].quantile(0.75),
            'Max': df_desnormalizado[var].max()
        }
        
        resultados.extend([stats_norm, stats_desnorm])
    
    return pd.DataFrame(resultados)

# Variables a comparar en el reporte global
variables = ['Edad', 'IMC', 'Trigliceridos', 'PAS', 'PAD']

# Generamos y mostramos reporte global
df_comparacion_global = comparar_global(df, df_desnormalizado, variables)
print("📊 Comparación Global (Normalizado vs Desnormalizado):")
display(df_comparacion_global.round(2))

📊 Comparación Global (Normalizado vs Desnormalizado):


,Variable,Estado,Media,Std,Min,P25,P50 (Mediana),P75,Max
0,Edad,Normalizado,-0.00,0.99,-2.94,-0.66,0.05,0.68,2.95
1,Edad,Desnormalizado,52.92,11.03,21.00,46.00,54.00,61.00,81.00
2,IMC,Normalizado,0.00,1.00,-2.97,-0.76,-0.05,0.75,2.83
3,IMC,Desnormalizado,30.35,5.76,17.00,25.80,30.70,34.60,45.60
4,Trigliceridos,Normalizado,0.00,1.00,-2.99,-0.80,0.01,0.79,2.83
5,Trigliceridos,Desnormalizado,192.85,56.66,90.00,143.39,182.41,243.14,300.00
6,PAS,Normalizado,-0.00,0.99,-2.81,-0.75,0.00,0.74,2.94
7,PAS,Desnormalizado,149.66,16.79,99.00,138.40,151.30,161.60,194.60
8,PAD,Normalizado,0.00,0.99,-2.90,-0.76,-0.01,0.73,2.97
9,PAD,Desnormalizado,91.49,9.45,64.30,84.90,92.40,98.30,116.70


In [18]:
def verificar_fila_a_fila_desnormalizacion(df_norm, df_desnorm, params_norm, cols, tolerancia=0.1, mostrar_resumen=True):
    """
    Verifica fila a fila si la desnormalización de cada valor es correcta,
    comparando el valor desnormalizado calculado (valor_normalizado*std+mean)
    con el valor real obtenido en df_desnorm.
    Retorna un DataFrame con la comparación y opcionalmente muestra un resumen.
    """
    resultados = []

    for idx, row in df_norm.iterrows():
        cohorte = row['Cohorte']
        for col in cols:
            valor_normalizado = row[col]
            std = params_norm[cohorte]['std'][cols.index(col)]
            media = params_norm[cohorte]['media'][cols.index(col)]

            valor_esperado = (valor_normalizado * std) + media
            valor_desnormalizado = df_desnorm.loc[idx, col]
            diferencia = abs(valor_esperado - valor_desnormalizado)

            resultados.append({
                'Índice': idx,
                'Cohorte': cohorte,
                'Variable': col,
                'Valor_Normalizado': valor_normalizado,
                'Esperado_Desnormalizado': valor_esperado,
                'Obtenido_Desnormalizado': valor_desnormalizado,
                'Diferencia': diferencia,
                'OK': diferencia < tolerancia
            })

    df_resultado = pd.DataFrame(resultados)
    n_total = len(df_resultado)
    n_errores = (~df_resultado['OK']).sum()
    n_correctos = n_total - n_errores

    if mostrar_resumen:
        if n_errores == 0:
            print(f"✅ Verificación completada: {n_total} comparaciones, 100% correctas.")
        else:
            print(f"❌ Verificación incompleta: {n_correctos}/{n_total} correctas.")
            print(f"    → {n_errores} comparaciones con error. Revisa el DataFrame.")

    return df_resultado

# Ejecutamos la verificación fila a fila
df_verif_filas = verificar_fila_a_fila_desnormalizacion(df, df_desnormalizado, params_norm, cols_to_normalize)

# Mostrar solo las filas con errores (si las hubiera)
if not df_verif_filas[df_verif_filas['OK'] == False].empty:
    display(df_verif_filas[df_verif_filas['OK'] == False].round(3))

✅ Verificación completada: 79745 comparaciones, 100% correctas.


In [19]:
df_desnormalizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15949 entries, 0 to 15948
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Año_Registro           15949 non-null  int64  
 1   Edad                   15949 non-null  float64
 2   IMC                    15949 non-null  float64
 3   Diabetes               15949 non-null  int64  
 4   Colesterol_Total       15949 non-null  float64
 5   Trigliceridos          15949 non-null  float64
 6   PAS                    15949 non-null  float64
 7   PAD                    15949 non-null  float64
 8   Cohorte                15949 non-null  object 
 9   HbA1c                  15949 non-null  float64
 10  Insulina               15949 non-null  float64
 11  LDL                    15949 non-null  float64
 12  HDL                    15949 non-null  float64
 13  Nivel_Estres           15949 non-null  object 
 14  Actividad_Fisica       15949 non-null  object 
 15  Ri

In [20]:
# Análisis descriptivo de las variables solicitadas en df_desnormalizado
variables_analisis = ['Edad', 'Trigliceridos', 'PAS', 'PAD', 'Insulina']

desc_stats = df_desnormalizado[variables_analisis].describe().T
desc_stats['missing'] = df_desnormalizado[variables_analisis].isnull().sum()
desc_stats = desc_stats[['count', 'missing', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]

print("📊 Análisis descriptivo de Edad, Trigliceridos, PAS, PAD, Insulina:")
display(desc_stats.round(2))

📊 Análisis descriptivo de Edad, Trigliceridos, PAS, PAD, Insulina:


,count,missing,mean,std,min,25%,50%,75%,max
Edad,15949.0,0,52.92,11.03,21.0,46.00,54.00,61.00,81.00
Trigliceridos,15949.0,0,192.85,56.66,90.0,143.39,182.41,243.14,300.00
PAS,15949.0,0,149.66,16.79,99.0,138.40,151.30,161.60,194.60
PAD,15949.0,0,91.49,9.45,64.3,84.90,92.40,98.30,116.70
Insulina,15949.0,0,82.13,56.60,0.0,34.95,76.73,124.27,294.32


## Interpretación Clínica de Resultados

A continuación se presenta una evaluación clínica básica de las variables principales del dataset, basada en sus estadísticas descriptivas.

| Variable     | Media    | Rango (min - max) | Comentarios Clínicos |
|--------------|----------|-------------------|---------------------|
| **Edad**     | 52.9 años| 21 - 81 años      | Edad típica en estudios de riesgo cardiovascular y diabetes tipo 2, cubriendo adultos jóvenes a personas mayores. |
| **Triglicéridos (mg/dL)** | 193      | 90 - 300           | Valores elevados respecto al rango normal (<150 mg/dL), frecuentes en pacientes con riesgo metabólico o diabetes. |
| **Presión Arterial Sistólica (PAS, mmHg)** | 150      | 99 - 194           | Media indica hipertensión grado 1 o más, común en poblaciones con riesgo cardiovascular. |
| **Presión Arterial Diastólica (PAD, mmHg)** | 91.5     | 64 - 117           | Valores elevados (>80 mmHg), indicativos de hipertensión diastólica frecuente en la muestra. |
| **Insulina (µU/mL)**        | 82       | 0 - 294            | Valores elevados respecto a rangos normales en ayunas (2-25 µU/mL), sugiriendo hiperinsulinemia y resistencia a la insulina asociadas a diabetes tipo 2. |

---

### Resumen

Los valores estadísticos del dataset son clínicamente plausibles y consistentes con una población de pacientes con riesgo metabólico, diabetes y enfermedad cardiovascular.  
No se observan valores aberrantes que indiquen errores de medición o entrada de datos.  

Esta información valida la calidad del dataset y su utilidad para análisis clínicos y epidemiológicos relacionados con estas patologías.



In [21]:
df_desnormalizado

,Año_Registro,Edad,IMC,Diabetes,Colesterol_Total,Trigliceridos,PAS,PAD,Cohorte,HbA1c,Insulina,LDL,HDL,Nivel_Estres,Actividad_Fisica,Riesgo_Cardiovascular,Sexo,Tipo_Diabetes
0,2016,53.0,30.464413,0,203.053597,228.381680,146.4,89.7,cardio,5.483777,10.602855,111.284292,71.456754,Alto,Moderado,Bajo,Hombre,No diabetes
1,2013,66.0,21.307639,0,214.198250,246.749168,151.8,87.2,cardio,5.212008,56.952216,124.064267,48.292897,Moderado,Sedentario,Bajo,Mujer,No diabetes
2,2017,68.0,32.672495,0,178.338545,214.823232,157.4,88.1,cardio,6.750564,92.875636,123.757918,55.659980,Moderado,Moderado,Bajo,Mujer,Diabetes latente
3,2014,60.0,41.167111,0,208.059981,242.028543,175.5,95.1,cardio,4.659614,188.637574,110.158158,54.487884,Moderado,Sedentario,Bajo,Mujer,Diabetes latente
4,2016,61.0,31.813888,1,260.781259,243.954935,164.6,93.9,cardio,7.436320,66.564019,120.410972,52.990636,Moderado,Sedentario,Riesgo Elevado,Mujer,Diabetes latente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15944,2017,40.0,27.900000,1,210.750000,134.360000,129.3,81.8,diabetes,5.840000,4.450000,96.230000,53.870000,Bajo,Moderado,Bajo,Hombre,Tipo 1
15945,2012,38.0,33.000000,1,226.150000,158.550000,135.6,90.3,diabetes,7.810000,79.930000,107.710000,43.000000,Alto,Moderado,Riesgo Elevado,Hombre,Tipo 2
15946,2012,46.0,25.900000,1,215.130000,115.980000,130.2,82.9,diabetes,7.370000,12.610000,103.980000,50.240000,Bajo,Activo,Bajo,Hombre,Tipo 1
15947,2016,63.0,21.400000,1,228.540000,144.610000,146.3,82.7,diabetes,8.100000,0.390000,108.890000,54.810000,Alto,Sedentario,Riesgo Elevado,Mujer,Tipo 1


In [22]:
# Guardar el df desnoramalizado
df_cardiabetes_desnorm = df_desnormalizado.copy()
df_cardiabetes_desnorm.to_csv("db_cardiabetes_desnorm.csv", index=False)